In [ ]:
import mediapipe as mp
import numpy as np

import cv2

In [ ]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    model_complexity=1,
)

In [ ]:
def is_all_fingers_closed(landmarks):
    # Fingers 4-8 (index to pinky) should be closed
    return all(landmarks[i].y > landmarks[i - 2].y for i in [8, 12, 16, 20])

def detect_left_to_right_movement(prev_x, current_x, threshold=20):
    if prev_x is not None and (current_x - prev_x) > threshold:
        return True
    return False

# Function to detect right-to-left movement
def detect_right_to_left_movement(prev_x, current_x, threshold=20):
    if prev_x is not None and (prev_x - current_x) > threshold:
        return True
    return False

def move_bot(direction):
    print(f"Moving bot {direction}")



In [ ]:
def are_fingers_close(landmarks, image_width, threshold=30):
    # Index finger tip (landmark 8) and middle finger tip (landmark 12)
    index_finger_tip = landmarks[8]
    middle_finger_tip = landmarks[12]
    
    # Calculate the Euclidean distance between the two landmarks in 2D
    distance = np.sqrt((index_finger_tip.x - middle_finger_tip.x) ** 2 +
                       (index_finger_tip.y - middle_finger_tip.y) ** 2)
    
    # Convert distance to pixel units by considering image dimensions
    distance_in_pixels = distance * image_width
    
    return distance_in_pixels < threshold

In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    draw= mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            image_height, image_width, _ = frame.shape
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            processed = hands.process(rgb_frame)
            
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
                
                if are_fingers_close(hand_landmarks.landmark, image_width) and not is_all_fingers_closed(landmarks):
                    index_finger_tip = hand_landmarks.landmark[8]
                    current_x, current_y = int(index_finger_tip.x * image_width), int(index_finger_tip.y * image_height)
                    
                    if current_x < image_width // 2:
                        cv2.putText(frame, 'Move Left', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                        move_bot("left")
                    else:
                        cv2.putText(frame, 'Move Right', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        move_bot("right")
                landmarks = hand_landmarks.landmark
                hand_center_x = int(landmarks[0].x * image_width)  # Assume hand center is roughly at landmark[0]
                    
                if is_all_fingers_closed(landmarks):
                        if hand_center_x < image_width // 2:
                            # All fingers closed on left hand
                            move_bot("backward")
                            cv2.putText(frame, 'Moving Backward', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        else:
                            # All fingers closed on right hand
                            move_bot("forward")
                            cv2.putText(frame, 'Moving Forward', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
main()